# District Heating Data

In [5]:
import json
from datetime import timedelta
import os

import pandas as pd
import polars as pl

from src.energy_forecast.config import RAW_DATA_DIR

In [29]:
dh_data_folder = RAW_DATA_DIR / "district_heating_data"
dh_data_folder_merge = dh_data_folder / "data"
dh_data_folder_first_part = dh_data_folder / "2024_01_29 Projekt KI-FW Data Export (2022_08-2024_01_29)" / "data"
dh_data_folder_second_part = dh_data_folder / "2024_05_14 Projekt KI-FW Data Export (2024_02_08-2024_05_14)"
file_path = dh_data_folder_first_part / "dump.csv"

In [4]:
df = pl.read_csv(file_path)
df

time,value,sensor_id,eco_u_id,data_provider_id
str,f64,str,str,str
"""2022-08-23T06:51:43.631Z""",0.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
"""2022-08-23T07:52:09.604Z""",0.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
"""2022-08-23T11:52:30.431Z""",0.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
"""2022-08-23T13:52:12.392Z""",0.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
"""2022-08-23T15:52:51.276Z""",3900.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
…,…,…,…,…
"""2024-01-29T07:32:35.347Z""",23377.73,"""63d3721c-f850-4eb0-98ce-1c16b3…","""a9644794-439b-401c-b879-8c0225…","""607e3978xesq"""
"""2024-01-29T08:32:48.301Z""",23379.03,"""63d3721c-f850-4eb0-98ce-1c16b3…","""a9644794-439b-401c-b879-8c0225…","""607e3978xesq"""
"""2024-01-29T09:32:20.318Z""",23379.96,"""63d3721c-f850-4eb0-98ce-1c16b3…","""a9644794-439b-401c-b879-8c0225…","""607e3978xesq"""


In [30]:
df_2 = pl.read_csv(dh_data_folder_second_part / "dump.csv")
df_2

time,value,sensor_id,eco_u_id,data_provider_id
str,f64,str,str,str
"""2024-02-08T00:39:56.224Z""",0.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
"""2024-02-08T01:40:14.166Z""",-100.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
"""2024-02-08T02:40:20.139Z""",-200.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
"""2024-02-08T03:40:42.080Z""",-100.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
"""2024-02-08T04:40:22.043Z""",0.0,"""d5209239-0c22-4a31-8bcc-0af1a0…","""c00c8cba-b6de-4c10-89c0-e92312…","""86237bdbtc1i"""
…,…,…,…,…
"""2024-05-14T03:18:08.646Z""",25703.69,"""63d3721c-f850-4eb0-98ce-1c16b3…","""a9644794-439b-401c-b879-8c0225…","""607e3978xesq"""
"""2024-05-14T04:18:32.647Z""",25704.53,"""63d3721c-f850-4eb0-98ce-1c16b3…","""a9644794-439b-401c-b879-8c0225…","""607e3978xesq"""
"""2024-05-14T05:18:04.935Z""",25705.19,"""63d3721c-f850-4eb0-98ce-1c16b3…","""a9644794-439b-401c-b879-8c0225…","""607e3978xesq"""


In [22]:
df.filter(pl.col("eco_u_id") == "0c9ad311-b86f-4371-a695-512ca49c70a7",
          pl.col("data_provider_id") == "bd95e281dopi",
          pl.col("sensor_id") == "70a9a928-5277-44e9-b2dc-a4b06c048e10")

time,value,sensor_id,eco_u_id,data_provider_id
str,f64,str,str,str
"""2022-09-30T10:19:50.926Z""",1.806548e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2022-09-30T11:20:18.905Z""",1.806572e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2022-09-30T12:20:37.938Z""",1.8066e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2022-09-30T13:20:52.883Z""",1.806629e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2022-09-30T15:20:34.869Z""",1.806684e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
…,…,…,…,…
"""2023-12-04T04:23:05.346Z""",2.101401e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2023-12-04T05:22:46.367Z""",2.101401e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2023-12-04T07:23:09.410Z""",2.101401e6,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""


In [31]:
df_2.filter(pl.col("eco_u_id") == "0c9ad311-b86f-4371-a695-512ca49c70a7",
          pl.col("data_provider_id") == "bd95e281dopi",
          pl.col("sensor_id") == "70a9a928-5277-44e9-b2dc-a4b06c048e10")

time,value,sensor_id,eco_u_id,data_provider_id
str,f64,str,str,str
"""2024-05-02T11:27:42.106Z""",0.0,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2024-05-02T11:28:42.084Z""",0.0,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2024-05-02T11:29:42.084Z""",0.0,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2024-05-02T12:29:16.089Z""",0.0,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2024-05-02T13:29:36.072Z""",0.0,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
…,…,…,…,…
"""2024-05-14T03:26:01.784Z""",0.0,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2024-05-14T04:26:10.713Z""",0.0,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""
"""2024-05-14T05:26:01.686Z""",0.0,"""70a9a928-5277-44e9-b2dc-a4b06c…","""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi"""


In [26]:
for file in os.listdir(dh_data_folder_first_part):
    filename = os.fsdecode(file)
    filename, file_extension = os.path.splitext(filename)
    print(filename)
    if file_extension == ".json":
        with open(dh_data_folder_first_part / file, "r", encoding="UTF-8") as f:
            eco_u_data = json.loads(f.read())
        for data_point in eco_u_data["datapoint_data"]:
            if data_point["title"] == "FW Wärmemenge":
                sensor_id = data_point["id"]
                print(sensor_id)
        eco_u_id, data_provider_id = filename.split(".")
        df_sensor = df.filter(pl.col("eco_u_id") == eco_u_id,
                              pl.col("data_provider_id") == data_provider_id,
                              pl.col("sensor_id") == sensor_id
                              ).sort(pl.col("time"))
        print(len(df_sensor))
        store_csv_path = dh_data_folder_merge / f"{filename}_1.csv"
        df_sensor.write_csv(store_csv_path)
        print("Created " + str(store_csv_path))

8f7b3862-a50d-44eb-8ac9-de0cf48a6bd2.8ab8b58beel3
a04d39d6-4772-4cb0-8359-5c9b4f8ca9a5
6147
Created /home/marja/PycharmProjects/energy-forecast-prototype/data/raw/district_heating_data/data/8f7b3862-a50d-44eb-8ac9-de0cf48a6bd2.8ab8b58beel3_1.csv
42d6efdc-d590-40b7-af9a-90121d04d3a9.0bcadf46b2ic
101016e7-4ef8-46fb-8a44-bdca27eeb39c
9873
Created /home/marja/PycharmProjects/energy-forecast-prototype/data/raw/district_heating_data/data/42d6efdc-d590-40b7-af9a-90121d04d3a9.0bcadf46b2ic_1.csv
c00c8cba-b6de-4c10-89c0-e923127acc4e.71758690flyq
98c3ccff-f40a-4d28-b08d-5ae0f3189789
7649
Created /home/marja/PycharmProjects/energy-forecast-prototype/data/raw/district_heating_data/data/c00c8cba-b6de-4c10-89c0-e923127acc4e.71758690flyq_1.csv
7bdbc8ee-00fb-4795-99cb-c1739fc128ad.5c852fb74usj
e8bbf035-5195-454e-a3d0-da9ae8f492c8
8417
Created /home/marja/PycharmProjects/energy-forecast-prototype/data/raw/district_heating_data/data/7bdbc8ee-00fb-4795-99cb-c1739fc128ad.5c852fb74usj_1.csv
d00d6502-a08d-45

In [35]:
for file in os.listdir(dh_data_folder_second_part):
    filename = os.fsdecode(file)
    filename, file_extension = os.path.splitext(filename)
    print(filename)
    if file_extension == ".json":
        with open(dh_data_folder_second_part / file, "r", encoding="UTF-8") as f:
            eco_u_data = json.loads(f.read())
        for data_point in eco_u_data["datapoint_data"]:
            if data_point["title"] == "FW Wärmemenge":
                sensor_id = data_point["id"]
                print(sensor_id)
        eco_u_id, data_provider_id = filename.split(".")
        df_sensor = df_2.filter(pl.col("eco_u_id") == eco_u_id,
                              pl.col("data_provider_id") == data_provider_id,
                              pl.col("sensor_id") == sensor_id
                              ).sort(pl.col("time"))
        print(len(df_sensor))
        store_csv_path = dh_data_folder_merge / f"{filename}_2.csv"
        df_sensor.write_csv(store_csv_path)
        print("Created " + str(store_csv_path))

8f7b3862-a50d-44eb-8ac9-de0cf48a6bd2.8ab8b58beel3
a04d39d6-4772-4cb0-8359-5c9b4f8ca9a5
1950
Created /home/marja/PycharmProjects/energy-forecast-prototype/data/raw/district_heating_data/data/8f7b3862-a50d-44eb-8ac9-de0cf48a6bd2.8ab8b58beel3_2.csv
42d6efdc-d590-40b7-af9a-90121d04d3a9.0bcadf46b2ic
101016e7-4ef8-46fb-8a44-bdca27eeb39c
2104
Created /home/marja/PycharmProjects/energy-forecast-prototype/data/raw/district_heating_data/data/42d6efdc-d590-40b7-af9a-90121d04d3a9.0bcadf46b2ic_2.csv
c00c8cba-b6de-4c10-89c0-e923127acc4e.71758690flyq
98c3ccff-f40a-4d28-b08d-5ae0f3189789
1616
Created /home/marja/PycharmProjects/energy-forecast-prototype/data/raw/district_heating_data/data/c00c8cba-b6de-4c10-89c0-e923127acc4e.71758690flyq_2.csv
7bdbc8ee-00fb-4795-99cb-c1739fc128ad.5c852fb74usj
e8bbf035-5195-454e-a3d0-da9ae8f492c8
1703
Created /home/marja/PycharmProjects/energy-forecast-prototype/data/raw/district_heating_data/data/7bdbc8ee-00fb-4795-99cb-c1739fc128ad.5c852fb74usj_2.csv
d00d6502-a08d-45

In [34]:
eco_u_data_merge = dict()
for file in os.listdir(dh_data_folder_first_part):
    filename = os.fsdecode(file)
    filename, file_extension = os.path.splitext(filename)
    print(filename)
    if file_extension == ".json":
        with open(dh_data_folder_first_part / file, "r", encoding="UTF-8") as f:
            eco_u_data = json.loads(f.read())
        eco_u_data_point = {filename: eco_u_data["economic_unit"]}
        eco_u_data_merge.update(eco_u_data_point)
json_object = json.dumps(eco_u_data_merge)
with open(dh_data_folder / "eco_u_ids.json", "w") as outfile:
    outfile.write(json_object)

8f7b3862-a50d-44eb-8ac9-de0cf48a6bd2.8ab8b58beel3
42d6efdc-d590-40b7-af9a-90121d04d3a9.0bcadf46b2ic
c00c8cba-b6de-4c10-89c0-e923127acc4e.71758690flyq
7bdbc8ee-00fb-4795-99cb-c1739fc128ad.5c852fb74usj
d00d6502-a08d-45df-99e3-7d8cd55200d1.b92753adeari
8e9b1544-434e-44a7-8049-8f2e4b14a819.f1af853af3qs
c1469372-a3c4-4870-808c-8039cb927db8.7ce81adfg4dr
edcafda6-fe6f-4ca3-bb3d-f0c5fbac8f4c.338908912uns
4edd9f9b-22a0-4932-a815-60c9a222826d.22c3f026bbcr
bc098a2e-0cc7-4f01-b6ad-9d647ae9f627.cec5ec7cspn2
4f36b3bd-337e-4b93-9333-c53a28d0c417.a540555dmo6i
5e2fd59d-603a-488b-a525-513541039c4a.47e757c2y9pw
fb5cc271-ae15-4f24-b9d5-30782bbc2089.42fe7d9fz6oj
44201958-2d6b-4952-956c-22ea951a6442.779fa186y9wj
2a5d5a37-a843-4621-b20a-a0c1a5e42e49.61f9a1bcgzpi
10af300b-a270-4e41-928d-e4048b2fdf00.2e5dd0d6bct7
2a5d5a37-a843-4621-b20a-a0c1a5e42e49.dc62aac1q9uf
72013db2-014c-4c4f-8eee-ce179b77ba6f.9f5b73d65fya
7bdbc8ee-00fb-4795-99cb-c1739fc128ad.efd4921fwjck
fb5cc271-ae15-4f24-b9d5-30782bbc2089.91cabaaewpqb


In [39]:
# merge daily values to one data file
df_1 = pl.read_csv(RAW_DATA_DIR / "district_heating_daily.csv")
df_2 = pl.read_csv(RAW_DATA_DIR / "district_heating_daily_2.csv")
merge_df = pl.concat([df_1, df_2]).sort(["eco_u_id", "data_provider_id", "date"])
merge_df

date,diff,qmbehfl,adresse,ort,plz,eco_u_id,data_provider_id,primary_energy,unit_code
str,f64,str,str,str,i64,str,str,str,str
"""2022-09-30""",364.0,null,"""Kielortring 14""","""Norderstedt""",22850,"""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""","""district heating""","""kwh"""
"""2022-10-01""",703.0,null,"""Kielortring 14""","""Norderstedt""",22850,"""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""","""district heating""","""kwh"""
"""2022-10-02""",334.0,null,"""Kielortring 14""","""Norderstedt""",22850,"""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""","""district heating""","""kwh"""
"""2022-10-03""",891.0,null,"""Kielortring 14""","""Norderstedt""",22850,"""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""","""district heating""","""kwh"""
"""2022-10-04""",661.0,null,"""Kielortring 14""","""Norderstedt""",22850,"""0c9ad311-b86f-4371-a695-512ca4…","""bd95e281dopi""","""district heating""","""kwh"""
…,…,…,…,…,…,…,…,…,…
"""2024-05-10""",50.0,null,"""Heidehofweg 99c""","""Norderstedt""",22850,"""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf""","""district heating""","""kwh"""
"""2024-05-11""",51.0,null,"""Heidehofweg 99c""","""Norderstedt""",22850,"""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf""","""district heating""","""kwh"""
"""2024-05-12""",45.0,null,"""Heidehofweg 99c""","""Norderstedt""",22850,"""fb684f25-a63d-4d3e-9277-6d759b…","""75febb50vqdf""","""district heating""","""kwh"""


In [40]:
merge_df.write_csv(RAW_DATA_DIR / "district_heating_daily_merge.csv")

In [45]:
merge_df.group_by(["eco_u_id", "data_provider_id"]).agg(pl.col("adresse"), pl.len())

eco_u_id,data_provider_id,adresse,len
str,str,list[str],u32
"""e7ad9b75-bc6c-4891-a8fd-45e393…","""199f74d54sol""","[""Heidehofweg 122a+b +124"", ""Heidehofweg 122a+b +124"", … ""Heidehofweg 122a+b +124""]",537
"""72013db2-014c-4c4f-8eee-ce179b…","""b21844a2dbvi""","[""Moorbekstraße 120"", ""Moorbekstraße 120"", … ""Moorbekstraße 120""]",549
"""fb5cc271-ae15-4f24-b9d5-30782b…","""f922d694rpnf""","[""Moorbekstraße 33"", ""Moorbekstraße 33"", … ""Moorbekstraße 33""]",601
"""72013db2-014c-4c4f-8eee-ce179b…","""9f5b73d65fya""","[""Moorbekstraße 120"", ""Moorbekstraße 120"", … ""Moorbekstraße 120""]",566
"""1657f5b3-fad0-4685-b56c-d57982…","""35e798b1sekb""","[""Moorbekstraße 15"", ""Moorbekstraße 15"", … ""Moorbekstraße 15""]",595
…,…,…,…
"""882f402f-daf8-4dfb-8af0-6fbd5d…","""6fb87ad8f6oi""","[""Waldstraße 63"", ""Waldstraße 63"", … ""Waldstraße 63""]",600
"""35d897c4-9486-41c1-be9b-0e1707…","""a304c6f03ois""","[""Moorbekstraße 43-49"", ""Moorbekstraße 43-49"", … ""Moorbekstraße 43-49""]",508
"""1657f5b3-fad0-4685-b56c-d57982…","""86c8052ag2fa""","[""Moorbekstraße 15"", ""Moorbekstraße 15"", … ""Moorbekstraße 15""]",593
